In [36]:
import pandas
import datetime
import glob
import bz2
import time

In [38]:
srchDir = "/sd-data/2015/mappot_ascii/north/"
srchFilePtrn = "2015010*.bz2"
cpcpSrchString = "> Potential:"
dtSrchString = "d "

In [43]:
def get_cpcp():
    # Loop over the files and collect datetime/cpcp info
    # for each 2 minute time interval!
    # arrays to store the data
    dtList = []
    cpcpList = []
    for _fName in glob.glob(srchDir + srchFilePtrn):
        # open the file
        print "working with---->", _fName
        _file = bz2.BZ2File(_fName)
        for _line in _file:
            if _line.startswith(dtSrchString):
                dtStr = _line.split("d ")[1].strip("\n")
                dtObj = datetime.datetime.strptime(dtStr,"%Y-%m-%d/%H:%M:%S")
                dtList.append( dtObj )
            if cpcpSrchString in _line:
                cpStr = _line.split("Drop=")[1]
                cpVal = cpStr.split(" kV")
                cpcpList.append( int(cpVal[0]) )
    return (dtList, cpcpList)

In [44]:
s1 = time.clock()
(dtList, cpcpList) = get_cpcp()
print time.clock() - s1

/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/usr/local/lib/python2.7/dist-packages/scipy/linalg/__init__.py:191: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *


76.399736


In [47]:
cpcpDF = pandas.DataFrame({
    "date" : dtList,
    "cpcp" : cpcpList
    }
    )
cpcpDF.memory_usage(deep=True).sum()/(1024.**2)

0.04401397705078125

In [51]:
cpcpDF.to_csv("../data/test.csv", index=False, columns=["date", "cpcp"])
cpcpDF.to_feather("../data/test.feather")